In [ ]:
# %load script/addLengthField.py
# addLengthField.py
# Purpose: Add a field containing polygon lengths to the shapefile.
# Usage: No arguments needed.
import arcpy
arcpy.env.workspace = 'C:/gispy/data/ch06'

inputFile = 'special_regionsCopy.shp'
fieldName = 'Length'
arcpy.AddField_management(inputFile, fieldName, 'FLOAT')
arcpy.CalculateField_management(inputFile, fieldName, '!shape.length!', 'PYTHON')


In [ ]:
# %load script/avgNearNeighbor.py
# Name: avgNearNeighbor.py
# Purpose: Analyze crime data to determine if spatial patterns are
#         statistically significant.

import arcpy

arcpy.env.workspace = 'C:/gispy/data/ch06'

annResult = arcpy.AverageNearestNeighbor_stats('points.shp', 'Euclidean Distance')

print 'Average Nearest Neighbor Output'
print 'Nearest neighbor ratio: {0}'.format(annResult.getOutput(0))
print 'z-score: {0}'.format(annResult.getOutput(1))
print 'p-value: {0}'.format(annResult.getOutput(2))
print 'Expected Mean Distance: {0}'.format(annResult.getOutput(3))
print 'Observed Mean Distance: {0}'.format(annResult.getOutput(4))


In [ ]:
# %load script/avgNearNeighbor2.py
# Name: avgNearNeighbor.py
# Purpose: Analyze crime data to determine if spatial patterns are
#         statistically significant.

import arcpy

arcpy.env.workspace = './data'

annResult = arcpy.AverageNearestNeighbor_stats('points.shp', 'Euclidean Distance')

print 'Average Nearest Neighbor Output'
print 'Nearest neighbor ratio: {0}'.format(annResult.getOutput(0))
print 'z-score: {0}'.format(annResult.getOutput(1))
print 'p-value: {0}'.format(annResult.getOutput(2))
print 'Expected Mean Distance: {0}'.format(annResult.getOutput(3))
print 'Observed Mean Distance: {0}'.format(annResult.getOutput(4))


In [ ]:
# %load script/buffer_clip.py
# buffer_clip.py (hard-coded version)
# Purpose: Buffer a zone and use it to clip another file
# Input: No arguments needed.

import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/gispy/data/ch06'
outDir = 'C:/gispy/scratch/'

# Set buffer params
fireDamage = 'special_regions.shp'
fireBuffer = outDir + fireDamage[:-4] + 'Buffer.shp'
bufferDist = '1 mile'

# Set clip params
park = 'park.shp'
clipOutput = outDir + park[:-4] + 'DamageBuffer.shp'

arcpy.Buffer_analysis(fireDamage, fireBuffer, bufferDist)
print '{0} created.'.format(fireBuffer)
arcpy.Clip_analysis(park, fireBuffer, clipOutput)
print '{0} created.'.format(clipOutput)


In [ ]:
# %load script/callInventory.py
# callInventory.py
# Purpose: Call the inventory tool.
import arcpy

arcpy.env.workspace = 'C:/gispy/sample_scripts/ch06'
inputDir = "C:/gispy/data/ch06/"
summary = 'summaryFile.txt'
arcpy.ImportToolbox('customTools.tbx')
arcpy.Inventory_custom(inputDir, 'SUMMARY_ONLY', summary)
print 'Summary text {0} created in {1}'.format(summary, inputDir)


In [ ]:
# %load script/computeRastEquation.py
# computeRastEquation.py
# Purpose: Calculate 5 * 'getty_rast' - 2

import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/gispy/data/ch06/rastSmall.gdb'
arcpy.CheckOutExtension('Spatial')

outRast1 = arcpy.sa.Times(5, 'dataRast')
outRast2 = arcpy.sa.Minus(outRast1, 2)
outRast2.save('equationRast')

print 'Output raster written in {0}'.format(arcpy.env.workspace)
del outRast1, outRast2


In [ ]:
# %load script/inventory.py
# inventory.py
# Purpose: Summarize the file content of a directory, listing the number
#   of each file type and more detail if requested.
# Usage: workspace, level of detail (SUMMARY_ONLY or DETAILED_INVENTORY)
# Example: C:/gispy/data/ch06 SUMMARY_ONLY summary.txt

import arcpy, sys, os, traceback

debug = 1
if debug:
    for i, v in enumerate(sys.argv):
        print 'Input {0}: {1}'.format(i, v)


def printMessages(message):
    '''Print in both the Geoprocessing Window and the Interpreter Window'''
    arcpy.AddMessage(message)
    print message


def printSummary(title, listDict):
    '''Report the summary'''
    message = '''\nSummary of {0}
-------------------------------------\n'''.format(title)
    for key in listDict:
        listLen = len(listDict[key])
        message = message + '{0} {1}s\n'.format(listLen, key)
    printMessages(message)
    return message


def printDetails(title, listDict):
    '''Report the directory contents'''
    message = '''\nFile types in {0}
-------------------------------------\n'''.format(title)
    for key in listDict:
        message = message + '{0}s: {1}\n'.format(key, listDict[key])
    printMessages(message)
    return message

outfile = os.path.join(sys.argv[1], sys.argv[3])
with open(outfile, 'w') as outf:
    try:
        arcpy.env.workspace = sys.argv[1]
        fcs = arcpy.ListFeatureClasses()

        if os.path.isdir(arcpy.env.workspace):
            fs = os.listdir(arcpy.env.workspace)
            fDict = {}
            for f in fs:
                desc = arcpy.Describe(f)
                fType = desc.dataType
                if fType in fDict:
                    fDict[fType].append(f)
                else:
                    fDict[fType] = [f]
            m = printSummary(arcpy.env.workspace, fDict)
            outf.write(m)
            if sys.argv[2] == 'DETAILED_INVENTORY':
                m = printDetails(arcpy.env.workspace, fDict)
                outf.write(m)
        else:
            m = 'Not a valid directory {0}'.format(arcpy.env.workspace)
            printMessages(m)
            outf.write(m)
    except IndexError:
        m = 'Not enough arguments.  Usage: workspace level of detail \
            (SUMMARY_ONLY or DETAILED_INVENTORY)'
        printMessages(m)
        outf.write(m)
    except:
        traceback.print_exc()
        printMessages('The file that broke it is named {0}'.format(f))
        outf.write(m)


In [ ]:
# %load script/script6_6.py
import arcpy
arcpy.env.workspace = './data'
myData = 'xyData.txt'
arcpy.MakeXYEventLayer_management(myData,'x','y','tmpLayer')
countRes = arcpy.GetCount_management('tmpLayer')
countRes.getOutput(0)
arcpy.CopyFeatures_management('tmpLayer', 'butter flies.shp')